In [ ]:
!pip install transformers4rec[pytorch,nvtabular] -U

In [ ]:
!pip install cudf-cu11==22.12 rmm-cu11==22.12 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cugraph-cu11==22.12 dask-cuda==22.12 dask-cudf-cu11==22.12  pylibcugraph-cu11==22.12 --extra-index-url=https://pypi.ngc.nvidia.com/
!pip install cuml-cu11==22.12 raft_dask_cu11==22.12 dask-cudf-cu11==22.12  pylibraft_cu11==22.12 ucx-py-cu11==0.29.0 --extra-index-url=https://pypi.ngc.nvidia.com


In [ ]:
import os
import glob

import torch 
import transformers4rec.torch as tr

from transformers4rec.torch.ranking_metric import NDCGAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory

/usr/local/lib/python3.9/dist-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


In [ ]:
# Define categorical and continuous columns to fed to training model
cat_features = ['product_id-list_seq', 
                     'category_id-list_seq']

cont_features = ['price_log_norm-list_seq', 
                     'relative_price_to_avg_categ_id-list_seq']

temp_features = ['et_dayofweek_sin-list_seq',
                 'et_dayofweek_cos-list_seq',
                 'product_recency_days_log_norm-list_seq']

from merlin_standard_lib import Schema

# Define schema object to pass it to the TabularSequenceFeatures class
SCHEMA_PATH ='/content/drive/MyDrive/dataset_rees46/processed_nvt/schema.pbtxt'
schema = Schema().from_proto_text(SCHEMA_PATH)
schema = schema.select_by_name(cat_features + cont_features + temp_features)

In [ ]:
schema

[{'name': 'product_id-list_seq', 'value_count': {'min': '2', 'max': '20'}, 'type': 'INT', 'int_domain': {'name': 'product_id', 'max': '166795', 'is_categorical': True}, 'annotation': {'tag': ['list', 'item_id', 'id', 'categorical', 'item'], 'comment': ['{"is_ragged": true, "embedding_sizes": {"dimension": 512.0, "cardinality": 166796.0}, "max_size": 0.0, "is_list": true, "start_index": 1.0, "num_buckets": null, "freq_threshold": 0.0, "cat_path": ".//categories/unique.product_id.parquet", "dtype_item_size": 64.0}']}}, {'name': 'category_id-list_seq', 'value_count': {'min': '2', 'max': '20'}, 'type': 'INT', 'int_domain': {'name': 'category_id', 'max': '625', 'is_categorical': True}, 'annotation': {'tag': ['categorical', 'list'], 'comment': ['{"dtype_item_size": 64.0, "num_buckets": null, "is_list": true, "embedding_sizes": {"dimension": 59.0, "cardinality": 626.0}, "freq_threshold": 0.0, "max_size": 0.0, "start_index": 1.0, "cat_path": ".//categories/unique.category_id.parquet", "is_ragg

In [ ]:
#Input 
sequence_length, d_model = 20, 320

# Define input module to process tabular input-features and to prepare masked inputs
inputs= tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=sequence_length,
    aggregation="concat",
    continuous_projection=d_model,
    d_output=d_model,
    masking="mlm",
)

In [ ]:
#import transformers4rec.config.transformer as hf

transformer_config = tr.AlbertConfig.build(
    d_model=d_model, 
    item_embedding_dim = 320,
    n_head=8, 
    n_layer=2, 
    total_seq_length=sequence_length, 
    stochastic_shared_embeddings_replacement_prob = 0.06, #regularization
    input_dropout = 0.1,
    dropout = 0.0, #regularization
    label_smoothing = 0.2, #regularization (proved to be useful in train/val accuracy)
    weight_decay = 9.565968888623912e-05, #regularization,
    item_id_embeddings_init_std = 0.11,
    mlm_probability = 0.6,
    eval_on_last_item_seq_only = True,
    mf_constrained_embeddings = True,
    layer_norm_featurewise = True,
    num_hidden_groups = 1,
    inner_group_num = 1
)

# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs,
    tr.MLPBlock([d_model]),
    tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the head for to next item prediction task 
head = tr.Head(
    body,
    [tr.NextItemPredictionTask(weight_tying=True,
                              metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True),  
                                       RecallAt(top_ks=[10, 20], labels_onehot=True)])]           
)

# Get the end-to-end Model class 
model = tr.Model(head)

In [ ]:
model 

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (_aggregation): ConcatFeatures()
          (to_merge): ModuleDict(
            (continuous_module): SequentialBlock(
              (0): ContinuousFeatures(
                (filter_features): FilterFeatures()
                (_aggregation): ConcatFeatures()
              )
              (1): SequentialBlock(
                (0): DenseBlock(
                  (0): Linear(in_features=5, out_features=320, bias=True)
                  (1): ReLU(inplace=True)
                )
              )
              (2): AsTabular()
            )
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (product_id-list_seq): Embedding(166796, 64, padding_idx=0)
                (category_id-list_seq): Embedding(626, 64, padding_idx=0)
              )
            )
  

In [ ]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
from transformers4rec.torch.utils.data_utils import MerlinDataLoader

#Set arguments for training 
training_args = T4RecTrainingArguments(
            output_dir="/content/drive/MyDrive/dataset_rees46/bert_cat+price+temp",
            max_sequence_length=20,
            data_loader_engine='merlin',
            num_train_epochs=10, 
            dataloader_drop_last=True,
            compute_metrics_each_n_steps = 1,
            per_device_train_batch_size = 192,
            per_device_eval_batch_size = 512,
            gradient_accumulation_steps = 1,
            learning_rate=0.0004904752786458524,
            report_to = [],
            logging_steps=200,
        )

In [ ]:
# Instantiate the T4Rec Trainer, which manages training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    schema=schema,
    compute_metrics=True,
)

In [ ]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/content/drive/MyDrive/dataset_rees46/sessions_by_day")

In [ ]:
%%time
start_time_window_index = 1
final_time_window_index = 30
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/test.parquet"))
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.train_dataset_or_path = train_paths
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    # Evaluate on the following day
    trainer.eval_dataset_or_path = eval_paths
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    wipe_memory()

********************
Launch training for day 1 are:
********************



/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 111936
  Num Epochs = 10
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 5830


Step,Training Loss
200,10.240500
400,9.143900
600,8.701300
800,8.526500
1000,8.383400
1200,8.103500
1400,8.048300
1600,7.947900
1800,7.697600
2000,7.747300


Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, onl

********************
Eval results for day 2 are:	

********************

 eval_/loss = 7.696527004241943
 eval_/next-item/ndcg_at_10 = 0.11714424192905426
 eval_/next-item/ndcg_at_20 = 0.13906420767307281
 eval_/next-item/recall_at_10 = 0.2135937511920929
 eval_/next-item/recall_at_20 = 0.3005468547344208
 eval_runtime = 2.6474
 eval_samples_per_second = 4834.921
 eval_steps_per_second = 9.443
********************
Launch training for day 2 are:
********************



...

********************
Eval results for day 29 are:	

********************

 eval_/loss = 6.339688301086426
 eval_/next-item/ndcg_at_10 = 0.20399215817451477
 eval_/next-item/ndcg_at_20 = 0.2319708913564682
 eval_/next-item/recall_at_10 = 0.3536718785762787
 eval_/next-item/recall_at_20 = 0.46421873569488525
 eval_runtime = 2.6365
 eval_samples_per_second = 4854.909
 eval_steps_per_second = 9.482
********************
Launch training for day 29 are:
********************



***** Running training *****
  Num examples = 107328
  Num Epochs = 10
  Instantaneous batch size per device = 192
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 1
  Total optimization steps = 5590


Step,Training Loss
200,6.008500
400,5.954500
600,5.995300
800,5.869000
1000,5.834700
1200,5.893800
1400,5.784800
1600,5.782700
1800,5.807100
2000,5.702800


Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/MyDrive/dataset_rees46/bert_cat+price+temp/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, onl

********************
Eval results for day 30 are:	

********************

 eval_/loss = 6.278590202331543
 eval_/next-item/ndcg_at_10 = 0.2014462798833847
 eval_/next-item/ndcg_at_20 = 0.2297505885362625
 eval_/next-item/recall_at_10 = 0.3546093702316284
 eval_/next-item/recall_at_20 = 0.4665624797344208
 eval_runtime = 2.6784
 eval_samples_per_second = 4778.903
 eval_steps_per_second = 9.334
CPU times: user 2h 54min 35s, sys: 2min 46s, total: 2h 57min 21s
Wall time: 3h 6min 1s


In [ ]:
print("Results:")
for key, value in  model.compute_metrics().items(): 
  print('%s: %s ' % (key, value.item()))

In [ ]:
with open("/content/drive/MyDrive/dataset_rees46/results.txt", 'a') as f: 
    f.write('\n')
    f.write('Bert with categ feat + price feat + temp feat results:')
    f.write('\n')
    for key, value in  model.compute_metrics().items(): 
        f.write('%s: %s' % (key, value.item()))